In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('../data/social_media_ad_optimization_raw.csv', delimiter=',') 
df.head()

,user_id,age,gender,location,interests,ad_id,ad_category,ad_platform,ad_type,impressions,clicks,conversion,time_spent_on_ad,day_of_week,device_type,engagement_score
0,U0001,58,M,USA,Food,A0001,Sportswear,Facebook,Image,3,0,0,3.38,Friday,Mobile,0.02
1,U0002,55,F,USA,Tech,A0002,Electronics,Facebook,Image,9,9,1,6.77,Saturday,Tablet,0.93
2,U0003,52,F,UK,Gaming,A0003,Luggage,Instagram,Image,13,12,1,13.26,Wednesday,Mobile,0.93
3,U0004,31,F,USA,Tech,A0004,Gadgets,Facebook,Video,14,5,0,24.41,Saturday,Desktop,0.28
4,U0005,52,M,India,Tech,A0005,Luggage,Instagram,Carousel,10,5,0,21.43,Monday,Tablet,0.35


**Key Engineered Features**

In [3]:
df["CTR"]=df["clicks"]/df["impressions"]
df["CVR"]=df["conversion"]/df["clicks"].replace(0, np.nan)
df["CVR"]=df["CVR"].fillna(0)

**Interaction Features**

In [4]:
df["age_platform_interaction"] = df["age"] * df["ad_platform"].map({'Facebook': 1, 'Instagram': 2, 'Twitter': 3, 'LinkedIn': 4})
df["age_time_spent_interaction"] = df["age"] * df["time_spent_on_ad"]

**Contextual Features**

In [5]:
df["is_weekend"]=df["day_of_week"].isin(['Saturday', 'Sunday']).astype(int)
df["is_mobile"]=df["device_type"].isin(['Mobile']).astype(int)
df["is_facebook"]=df["ad_platform"].isin(['Facebook']).astype(int)
df["is_instagram"]=df["ad_platform"].isin(['Instagram']).astype(int)

df.head()

,user_id,age,gender,location,interests,ad_id,ad_category,ad_platform,ad_type,impressions,clicks,conversion,time_spent_on_ad,day_of_week,device_type,engagement_score,CTR,CVR,age_platform_interaction,age_time_spent_interaction,is_weekend,is_mobile,is_facebook,is_instagram
0,U0001,58,M,USA,Food,A0001,Sportswear,Facebook,Image,3,0,0,3.38,Friday,Mobile,0.02,0.000000,0.000000,58,196.04,0,1,1,0
1,U0002,55,F,USA,Tech,A0002,Electronics,Facebook,Image,9,9,1,6.77,Saturday,Tablet,0.93,1.000000,0.111111,55,372.35,1,0,1,0
2,U0003,52,F,UK,Gaming,A0003,Luggage,Instagram,Image,13,12,1,13.26,Wednesday,Mobile,0.93,0.923077,0.083333,104,689.52,0,1,0,1
3,U0004,31,F,USA,Tech,A0004,Gadgets,Facebook,Video,14,5,0,24.41,Saturday,Desktop,0.28,0.357143,0.000000,31,756.71,1,0,1,0
4,U0005,52,M,India,Tech,A0005,Luggage,Instagram,Carousel,10,5,0,21.43,Monday,Tablet,0.35,0.500000,0.000000,104,1114.36,0,0,0,1


In [6]:
df.describe()

,age,impressions,clicks,conversion,time_spent_on_ad,engagement_score,CTR,CVR,age_platform_interaction,age_time_spent_interaction,is_weekend,is_mobile,is_facebook,is_instagram
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,39.410000,8.282000,4.520000,0.470000,14.072300,0.520220,0.539864,0.166738,59.610000,545.393700,0.316000,0.338000,0.492000,0.508000
std,12.400219,4.217351,3.727383,0.499599,9.088768,0.306296,0.340362,0.273606,28.225152,395.170033,0.465379,0.473502,0.500437,0.500437
min,18.000000,1.000000,0.000000,0.000000,0.460000,0.000000,0.000000,0.000000,18.000000,8.280000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,4.750000,1.000000,0.000000,4.990000,0.270000,0.262500,0.000000,39.000000,202.380000,0.000000,0.000000,0.000000,0.000000
50%,39.000000,9.000000,4.000000,0.000000,13.285000,0.550000,0.533333,0.000000,52.500000,460.785000,0.000000,0.000000,0.000000,1.000000
75%,51.000000,12.000000,7.000000,1.000000,21.732500,0.772500,0.857143,0.200000,80.000000,781.815000,1.000000,1.000000,1.000000,1.000000
max,60.000000,15.000000,15.000000,1.000000,29.950000,1.000000,1.000000,1.000000,120.000000,1691.280000,1.000000,1.000000,1.000000,1.000000


**PCA-based Engagement Score**

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA

In [12]:
X_engg = df[["clicks","time_spent_on_ad"]]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_engg)

pca = PCA(n_components=1)
df["engagement_pca"] = pca.fit_transform(X_scaled)
df.head()

,user_id,age,gender,location,interests,ad_id,ad_category,ad_platform,ad_type,impressions,clicks,conversion,time_spent_on_ad,day_of_week,device_type,engagement_score,CTR,CVR,age_platform_interaction,age_time_spent_interaction,is_weekend,is_mobile,is_facebook,is_instagram,engagement_pca
0,U0001,58,M,USA,Food,A0001,Sportswear,Facebook,Image,3,0,0,3.38,Friday,Mobile,0.02,0.000000,0.000000,58,196.04,0,1,1,0,-1.691024
1,U0002,55,F,USA,Tech,A0002,Electronics,Facebook,Image,9,9,1,6.77,Saturday,Tablet,0.93,1.000000,0.111111,55,372.35,1,0,1,0,0.282045
2,U0003,52,F,UK,Gaming,A0003,Luggage,Instagram,Image,13,12,1,13.26,Wednesday,Mobile,0.93,0.923077,0.083333,104,689.52,0,1,0,1,1.357161
3,U0004,31,F,USA,Tech,A0004,Gadgets,Facebook,Video,14,5,0,24.41,Saturday,Desktop,0.28,0.357143,0.000000,31,756.71,1,0,1,0,0.896229
4,U0005,52,M,India,Tech,A0005,Luggage,Instagram,Carousel,10,5,0,21.43,Monday,Tablet,0.35,0.500000,0.000000,104,1114.36,0,0,0,1,0.664153


In [13]:
df.to_csv("../data/processed_data_with_pca.csv", index=False)
